In [1]:
# reformat data (even when there is duplicate)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, time

import warnings
warnings.filterwarnings('ignore')

In [11]:
starting_dir = os.getcwd()
print ("starting_dir:" + str(starting_dir))

# mac - phase I & II - morpho
complete_file_path = '/Users/kimd999/research/projects/toxicity/per_each_data/Phase_I_II/input/morpho/tall/zf_morphology_data_335_chemicals_2020DEC16.csv'

morph_data = pd.read_csv(complete_file_path, header = 0)
pd.set_option('display.max_columns', None)

starting_dir:/Users/kimd999/research/script_not_in_dropbox/srpAnalytics/analysis/latest/1_reformat/all_targets/morpho


In [12]:
morph_data.head()

,chemical.id,bottle.id,conc,plate.id,well,date,endpoint,value
0,53,C96423,0.0,9289,A06,20151105,MO24,1.0
1,53,C96423,0.0,9289,B06,20151105,MO24,0.0
2,53,C96423,0.0,9289,C06,20151105,MO24,0.0
3,53,C96423,0.0,9289,D06,20151105,MO24,0.0
4,53,C96423,0.0,9289,E06,20151105,MO24,0.0


In [6]:
# just TX part
#TX_bottle_id_morph_data = morph_data[morph_data['bottle.id'].str.contains("^TX")]
#TX_bottle_id_morph_data = morph_data[morph_data['bottle.id'].str.contains("TX$")]
#TX_bottle_id_morph_data.head()

In [10]:
#new_morph_data = morph_data.loc[morph_data['chemical.id'] == 197,['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']]
#new_morph_data.head()

In [11]:
#morph_data_w_end = new_morph_data.loc[new_morph_data['endpoint'] == 'MO24',['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']]
#morph_data_w_end.head()

In [6]:
np.sum(morph_data['value'] == 1)

46675

In [7]:
np.sum(morph_data['value'] == 0) # -> 82%

1321119

In [8]:
np.sum(np.isnan(morph_data['value']))

233006

In [9]:
morph_data.shape

(1600800, 8)

In [13]:
# Keep only relevant columns
columns_to_keep = ['chemical.id', 'conc', 'bottle.id', 'plate.id', 'well', 'endpoint', 'value'] 
# 'bottle.id' is added since Lisa wants to use it to join with behavioral data

morph_data_select = morph_data.loc[:,columns_to_keep]
morph_data_select.head()

,chemical.id,conc,bottle.id,plate.id,well,endpoint,value
0,53,0.0,C96423,9289,A06,MO24,1.0
1,53,0.0,C96423,9289,B06,MO24,0.0
2,53,0.0,C96423,9289,C06,MO24,0.0
3,53,0.0,C96423,9289,D06,MO24,0.0
4,53,0.0,C96423,9289,E06,MO24,0.0


In [14]:
print ("<before dropping duplicates> len(morph_data_select):" + str(len(morph_data_select)))
morph_data_select = morph_data_select.drop_duplicates()
print ("<after dropping duplicates>  len(morph_data_select):" + str(len(morph_data_select)))

len(morph_data_select):1600800
<after dropping duplicates> len(morph_data_select):1600800


In [16]:
start_time = time.time()

reformatted = pd.DataFrame()

total_number_of_unique_chemicals = 0
total_number_of_chemical_plate_well = 0


#full_devel = "full"
full_devel = "devel"

if (full_devel == "full"):
    chemical_id_from_here = np.unique(morph_data_select['chemical.id'])
else:
    chemical_id_from_here = np.unique([53])
    
    
for chemical_index in chemical_id_from_here:
    total_number_of_unique_chemicals += 1
#    if (chemical_index != 746) and (chemical_index != 1171):
 #       continue
    print("\nchemical_index:" + str(chemical_index))        
    
    morph_data_chemical = morph_all_data_select.loc[morph_all_data['chemical.id'] == chemical_index,:]
    #    print ("\ntotal_number_of_unique_chemicals:" + str(total_number_of_unique_chemicals))

    # Append chemical_plate_well as a unique identifier
    morph_data_chemical.insert(0, 'chemical_plate_well', morph_data_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))

    #print ("len(morph_data_chemical.chemical_plate_well):" + str(len(morph_data_chemical.chemical_plate_well)))    
    for cpw in np.unique(morph_data_chemical.chemical_plate_well):
        total_number_of_chemical_plate_well += 1
        temp_df = morph_data_chemical.loc[morph_data_chemical.chemical_plate_well == cpw,:]
        temp_df_grouped = temp_df.groupby(['chemical.id', 'plate.id', 'well'])
        #print ("temp_df_grouped:" + str(temp_df_grouped)) #<pandas.core.groupby.generic.DataFrameGroupBy object at 0x1292f0190>

        for name, group in temp_df_grouped:
#            if(len(group.endpoint) == 23):
                try:
                    temp = pd.DataFrame(
                        {
                        'chemical.id': np.unique(temp_df['chemical.id']),
                        'plate.id': np.unique(temp_df['plate.id']),
                        'well': np.unique(temp_df['well']),
                        'chemical_plate_well': np.unique(temp_df['chemical_plate_well']),
                        'conc': np.unique(temp_df['conc']),
                        'AXIS': temp_df.value[temp_df.endpoint == 'AXIS'].values,
                        'BRAI': temp_df.value[temp_df.endpoint == 'BRAI'].values,
                        'CFIN': temp_df.value[temp_df.endpoint == 'CFIN'].values,
                        'CIRC': temp_df.value[temp_df.endpoint == 'CIRC'].values,
                        'DNC_': temp_df.value[temp_df.endpoint == 'DNC_'].values,
                        'DP24': temp_df.value[temp_df.endpoint == 'DP24'].values,
                        'EYE_': temp_df.value[temp_df.endpoint == 'EYE_'].values,
                        'JAW_': temp_df.value[temp_df.endpoint == 'JAW_'].values,
                        'MO24': temp_df.value[temp_df.endpoint == 'MO24'].values,
                        'MORT': temp_df.value[temp_df.endpoint == 'MORT'].values,
                        'NC24': temp_df.value[temp_df.endpoint == 'NC24'].values,
                        'NC__': temp_df.value[temp_df.endpoint == 'NC__'].values,
                        'OTIC': temp_df.value[temp_df.endpoint == 'OTIC'].values,
                        'PE__': temp_df.value[temp_df.endpoint == 'PE__'].values,
                        'PFIN': temp_df.value[temp_df.endpoint == 'PFIN'].values,
                        'PIG_': temp_df.value[temp_df.endpoint == 'PIG_'].values,
                        'SM24': temp_df.value[temp_df.endpoint == 'SM24'].values,
                        'SNOU': temp_df.value[temp_df.endpoint == 'SNOU'].values,
                        'SOMI': temp_df.value[temp_df.endpoint == 'SOMI'].values,
                        'SWIM': temp_df.value[temp_df.endpoint == 'SWIM'].values,
                        'TRUN': temp_df.value[temp_df.endpoint == 'TRUN'].values,
                        'TR__': temp_df.value[temp_df.endpoint == 'TR__'].values,
                        'YSE_': temp_df.value[temp_df.endpoint == 'YSE_'].values,
                        }
                            )
                    reformatted = pd.concat([reformatted, temp])
                except:
                    print ("typical processing didn't work")
                    #print ("len(group.endpoint):" + str(len(group.endpoint)))
                    #print ("chemical_plate_well:" + str(cpw))

                    
print ("\ntotal_number_of_unique_chemicals:" + str(total_number_of_unique_chemicals))
print ("total_number_of_chemical_plate_well:" + str(total_number_of_chemical_plate_well))


######### automatically fix small manual mistakes ##########
reformatted.loc[(reformatted['chemical.id'] == 746) \
                & (reformatted['plate.id'] == 12953) \
                & (reformatted['well'] == "E03"), 'MORT'] = 0

reformatted.loc[(reformatted['chemical.id'] == 1171) \
                & (reformatted['plate.id'] == 15205) \
                & (reformatted['well'] == "D02"), 'MORT'] = 0

       
end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("Done, it took:"+str(time_took)) # took 13 minutes for 344 chemicals


chemical_index:53

total_number_of_unique_chemicals:1
total_number_of_chemical_plate_well:192
Done, it took:2.0 seconds


In [17]:
print (reformatted.head())
print ("reformatted.shape:" + str(reformatted.shape))

   chemical.id plate.id well chemical_plate_well  conc  AXIS  BRAI  CFIN  \
0           53     9289  A01         53_9289_A01  50.0   NaN   NaN   NaN   
0           53     9289  A02         53_9289_A02  35.6   0.0   0.0   0.0   
0           53     9289  A03         53_9289_A03  11.2   0.0   0.0   0.0   
0           53     9289  A04         53_9289_A04   5.0   1.0   0.0   0.0   
0           53     9289  A05         53_9289_A05   1.0   0.0   0.0   0.0   

   CIRC  DNC_  DP24  EYE_  JAW_  MO24  MORT  NC24  NC__  OTIC  PE__  PFIN  \
0   NaN   0.0   0.0   NaN   NaN   0.0   1.0   0.0   NaN   NaN   NaN   NaN   
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0   0.0   0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   1.0   
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   PIG_  SM24  SNOU  SOMI  SWIM  TRUN  TR__  YSE_  
0   NaN   0.0   NaN   NaN   

In [18]:
reformatted_DNC_0 = pd.DataFrame()
reformatted_DNC_0 = reformatted.loc[reformatted['DNC_'] == 0.0]
print ("reformatted_DNC_0.shape:" + str(reformatted_DNC_0.shape))

reformatted_DNC_0.shape:(192, 28)


In [19]:
reformatted_DNC_empty = pd.DataFrame()
reformatted_DNC_empty = reformatted.loc[reformatted['DNC_'] == '']
print ("reformatted_DNC_empty.shape:" + str(reformatted_DNC_empty.shape))

reformatted_DNC_empty.shape:(0, 28)


In [20]:
#'''
nan_count = 0
zero_count = 0
one_count = 0
total_count = 0
for (columnName, columnData) in reformatted.iteritems(): 
    if (columnName == "chemical.id") or (columnName == "plate.id") or (columnName == "well") or (columnName == "chemical_plate_well") or (columnName == "conc"):
        continue
#    print('Colunm Name : ', columnName) 
#    print('Column Contents : ', columnData.values[i])
#    print (len(columnData.values))
    for i in range(len(columnData.values)):
        total_count += 1
        if (str(columnData.values[i]) == "nan"):
            nan_count += 1
        elif (str(columnData.values[i]) == "0.0"):
            zero_count += 1
        elif (str(columnData.values[i]) == "1.0"):
            one_count += 1
#        print('Column Contents : ', columnData.values[i])
print ("nan_count:" + str(nan_count))
print ("zero_count:" + str(zero_count))
print ("one_count:" + str(one_count))
print ("total_count:" + str(total_count))
#'''

nan_count:156
zero_count:4188
one_count:72
total_count:4416


In [18]:
#pd.set_option('display.max_columns', None)

In [21]:
#reformatted.to_csv('344_zf_morphology_data_phase_1_and_2_-_2020JUNE25_wide_DNC_all.csv',index=False)

output_complete_file_path = complete_file_path[:-4] + "_wide_DNC_0_" \
                            + str(full_devel) + ".csv"
display ('output_complete_file_path:', str(output_complete_file_path))

reformatted_DNC_0.to_csv(output_complete_file_path,index=False)

#reformatted_DNC_empty.to_csv('344_zf_morphology_data_phase_1_and_2_-_2020JUNE25_wide_DNC_empty.csv',index=False)

'output_complete_file_path:'

'/Users/kimd999/research/projects/toxicity/per_each_data/Phase_I_II/input/morpho/tall/zf_morphology_data_335_chemicals_2020DEC16_wide_DNC_0_devel.csv'

In [20]:
#len(np.unique(reformatted['chemical.id']))

In [26]:
#reformatted.shape

In [29]:
#display(reformatted)